In [1]:
import sys

sys.path.insert(0,'../sib/')
import csv
import sib
import os
import numpy as np
import pandas as pd
import sklearn.metrics as mm
# sir_inference imports
from sir_model import FastProximityModel, patient_zeros_states
from ranking import csr_to_list
import os.path
from os import path
N=500000
#N = 5000
#N = 100
## new try with 100 spreaders
N_patient_zero = 200;
#N_patient_zero = 1;
lamb_load = 0.05;
lamb = 0.05
mu = 0.02;
scale=1.0; # Easy Case
T=100;
seed=2;
location="networks"
# SIR parameters
t1 = 10;
#seed=int();

##################### pruning fraction, per day
pru = 0.81 # ~ 0.90 AF
#pru = 0.64 # ~ 0.80 AF
#pru = 0.49 # ~ 0.70 AF
#pru = 0.36 # ~ 0.60 AF


Bad key "text.kerning_factor" on line 4 in
/opt/anaconda3/envs/bigdatalab_cpu_202101/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
## import sys
from time import time
import numpy as np
import pandas as pd
import os.path
from os import path
from scenario import Scenario
from sir_model import EpidemicModel, patient_zeros_states, symptomatic_individuals
from ranking import RANKINGS
log_fname="interactions_proximity_N%dK_s%.1f_T%d_lamb%.2f_s%d.csv"%(N/1000,scale,T,lamb_load,seed)
csv_file=location+"/"+log_fname
print("seed = %d"%seed, flush=True)
np.random.seed(seed);
if path.exists(csv_file): print("Loading model from %s"%csv_file)
else:
    print("Could not find the file! Was looking for \n "+csv_file+"\n Bye-Bye")
    sys.exit()
print("Load Proximity model", flush=True)
tic = time()
model = EpidemicModel(initial_states=np.zeros(N), x_pos=np.zeros(N), y_pos=np.zeros(N))
model.load_transmissions(csv_file, new_lambda = lamb)
model.recover_probas = mu*np.ones(N)
print(f"Loading took {time()-tic:.1f}s",flush=True)
model.initial_states = patient_zeros_states(N, N_patient_zero)
model.time_evolution(model.recover_probas, model.transmissions, print_every=50)
t_max = len(model.transmissions)
print("Save plain dynamics", flush=True)
db = pd.DataFrame()
db["S"] = np.sum(model.states==0,axis=1)
db["I"] = np.sum(model.states==1,axis=1)
db["R"] = np.sum(model.states==2,axis=1)
db.to_csv("csv/Proximity_N%dK_T%d_s1_pz%d_mu%.2f_l%.2f_seed%d.csv"%(N/1000,T,N_patient_zero,mu,lamb,seed),
          index=False, sep="\t")
del db
model.initial_states = model.states[t1]
model.states = model.states[t1:]
model.transmissions = model.transmissions[t1:]

    

seed = 2
Loading model from networks/interactions_proximity_N500K_s1.0_T100_lamb0.05_s2.csv
Load Proximity model
Loading transmissions from networks/interactions_proximity_N500K_s1.0_T100_lamb0.05_s2.csv
Loading took 479.1s
Running SIR simulation
t = 0 / 100
t = 50 / 100
Save plain dynamics


In [3]:
from scipy import sparse
model.pruned_transmissions = []
for t in range(0,len(model.transmissions)):
    A = model.transmissions[t].nonzero()
    row = A[0]
    col = A[1]
    keep = np.random.permutation(len(row))
    keep = keep[0:int(pru*len(row))]
    print(len(row), len(keep), len(keep)/len(row))    
    B = sparse.csr_matrix((lamb * np.ones(len(keep),), (row[keep], col[keep])), shape = (N,N))
    model.pruned_transmissions.append(B)
    



3002806 2432272 0.8099997136012117
3005692 2434610 0.809999826994915
3002332 2431888 0.8099996935715303
3002236 2431811 0.8099999467063882
3005744 2434652 0.8099997870743483
3000708 2430573 0.809999840037751
3008408 2436810 0.8099998404471734
3003836 2433107 0.8099999467347752
3006666 2435399 0.809999847006618
3004760 2433855 0.8099998003168306
3006380 2435167 0.809999733899241
3003198 2432590 0.8099998734682162
3003644 2432951 0.8099997869254811
3005308 2434299 0.8099998402825933
3005410 2434382 0.8099999667266695
3001274 2431031 0.8099996867996724
3006130 2434965 0.8099999002039167
3000652 2430528 0.8099999600086915
3004430 2433588 0.8099999001474489
3008372 2436781 0.8099998936301761
2999806 2429842 0.8099997133147944
3009136 2437400 0.8099999468285913
3002596 2432102 0.8099997468856949
3004544 2433680 0.8099997869893069
3002088 2431691 0.8099999067315815
3001604 2431299 0.8099999200427505
3006036 2434889 0.8099999467737579
3004980 2434033 0.8099997337752664
3002674 2432165 0.809999

In [4]:
tau = 5
# trac parameters
trac_tau = tau;
trac_lambd = lamb;
# MF parameters
MF_tau = tau;
MF_delta = 15;
# observation parameters
n_ranking = 1500
p_untracked=0
#seed = int(sys.argv[2]);
#seeds for running [32,123,456]
#seed=int(sys.argv[1]);
################################################

intervention_options=dict(quarantine_time=T)
observation_options=dict(n_random=0,n_infected=0,n_ranking=n_ranking, p_symptomatic=0.5, tau=5, p_untracked=p_untracked)

In [ ]:
import imp
import scenario
imp.reload(scenario)
from scenario import Scenario
import sib
import bp_ranking
imp.reload(bp_ranking)
from bp_ranking import bp_ranker_class

mu_r = np.log(1 + mu)
win = 21
#bp_tau = 10
name_csv = "csv/Proximity_N%dK_T%d_s1_ti%d_pz%d_mu%.2f_l%.2f_seed%d_obs%d_bp_no_tau_win%d_pru%.2lf.csv"%(N/1000, T,t1,N_patient_zero,mu,lamb,seed,n_ranking,win,pru)
prob_seed=1/N
prob_sus = 0.55
pseed = prob_seed / (2 - prob_seed)
psus = prob_sus * (1 - pseed)
pautoinf = 1e-10

bp_ranker = bp_ranker_class(params = sib.Params(
                                 prob_r = sib.Exponential(mu=mu_r),
                                 prob_i = sib.Uniform(p=lamb),
                                 pseed = pseed,
                                 psus = psus,
                                 pautoinf = pautoinf,
                                 fn_rate = 0.001,
                                 fp_rate = 0.001),
                                 
                 maxit0 = 30,
                 maxit1 = 30,
                 tol = 1e-3,
                 memory_decay = 0.1,
                 window_length = win
                            #tau=bp_tau
                           )
bp_ranker.init(N, T)
bp_ranker.__name__ = "bp"
scenario_bp = Scenario( model, seed=seed+1, 
    ranking_options=dict(ranking=bp_ranker.step_scenario, lamb = 1.0),
    observation_options=observation_options,
    intervention_options=intervention_options,
                         save_csv = name_csv
)
scenario_bp.run(t_max-t1, print_every = 1)
print("Save bp strategy", flush=True)
scenario_bp.counts.to_csv(name_csv,
          index=False, sep="\t")


Using step_scenario to rank
Using seed=3
Selecting 0/500000 untracked individuals

SCENARIO BP
t = 0 / 90
state: S:497798, I:2082, R:120 I+R:2202
obser: S:1494, I:6, R:0, I+R:6
free birds: 2076
obs_symp:0(0), obs_rank:6(6)

SCENARIO BP
Observations: 1 obs 1500 1500 0
sib.iterate(damp=0):  2/30 5.748e-07/0.001    
sib.iterate(damp=0.5):  0/30 4.172e-08/0.001    
winBP: (S,I,R): (499998.9, 1.0, 0.0), seeds: 0.9, ll: -298961.1
t = 1 / 90
state: S:497325, I:2526, R:149 I+R:2675
obser: S:2984, I:16, R:0, I+R:16
free birds: 2510
obs_symp:0(0), obs_rank:10(16)

SCENARIO BP
Observations: 2 obs 1500 1500 0
sib.iterate(damp=0):  2/30 3.341e-04/0.001    
sib.iterate(damp=0.5):  0/30 6.099e-05/0.001    
winBP: (S,I,R): (499998.7, 1.2, 0.1), seeds: 0.8, ll: -299031.7
t = 2 / 90
state: S:496813, I:3004, R:183 I+R:3187
obser: S:4310, I:171, R:19, I+R:190
free birds: 2834
obs_symp:0(0), obs_rank:155(171)

SCENARIO BP
Observations: 3 obs 1500 1500 0
sib.iterate(damp=0):  29/30 9.335e-01/0.001        
s

In [ ]:
import imp
import scenario
imp.reload(scenario)
from scenario import Scenario
##### TRACING UP TO SECOND NN #######
scenario_trac_2nd = Scenario( model, seed=seed+1, 
    ranking_options=dict(ranking=RANKINGS["tracing2nd"], tau=trac_tau, lamb=trac_lambd),
    observation_options=observation_options,
    intervention_options=intervention_options
)
scenario_trac_2nd.run(t_max-t1, print_every = 1)
print("Save tracing sec NN strategy", flush=True)
scenario_trac_2nd.counts.to_csv("csv/Proximity_N%dK_T%d_s1_ti%d_pz%d_mu%.2f_l%.2f_seed%d_obs%d_trac2nd_t%d_pru%.2lf.csv"%(N/1000,T,t1,N_patient_zero,mu,lamb,seed,n_ranking,trac_tau,pru),
                  index=False, sep="\t")
#scenario_trac_2nd.save("csv/Proximity_N%dK_T%d_s1_ti%d_pz%d_mu%.2f_l%.2f_seed%d_obs%d_trac2nd_t%d_all"%(N/1000,T,t1,N_patient_zero,mu,lamb,seed,n_ranking,trac_tau))

del scenario_trac_2nd
print("End seed", flush=True)

In [ ]:
##### MF SCENARIO #######
import imp
import scenario
imp.reload(scenario)
from scenario import Scenario
import ranking
imp.reload(ranking)
from ranking import RANKINGS
scenario_MF = Scenario(
    model, seed=seed+1, 
    ranking_options=dict(ranking=RANKINGS["backtrack"], 
                         algo="MF", init="all_S", tau=MF_tau, delta=MF_delta,mu=mu,lamb=lamb), 
    observation_options=observation_options,
    intervention_options=intervention_options,
)
scenario_MF.run(t_max-t1, print_every = 1)
print("Save MF strategy", flush=True)
scenario_MF.counts.to_csv("csv/Proximity_N%dK_T%d_s1_ti%d_pz%d_mu%.2f_l%.2f_seed%d_obs%d_MF_t%d_d%d_pru%.2lf.csv"%(N/1000,T,t1,N_patient_zero,mu,lamb,seed,n_ranking,MF_tau,MF_delta,pru),
                  index=False, sep="\t")
#scenario_MF.save("csv/Proximity_N%dK_T%d_s1_ti%d_pz%d_mu%.2f_l%.2f_seed%d_obs%d_MF_t%d_d%d_all"%(N/1000,T,t1,N_patient_zero,mu,lamb,seed,n_ranking,MF_tau,MF_delta))

del scenario_MF
# 1h01min per round
print("End seed", flush=True)

In [ ]:
import imp
import scenario
imp.reload(scenario)
from scenario import Scenario
##### RANDOM SCENARIO #######
scenario_rnd = Scenario(
    model, seed=seed+1, 
    ranking_options=dict(ranking=RANKINGS["random"]),
    observation_options=observation_options,
    intervention_options=intervention_options
)
scenario_rnd.run(t_max-t1,  print_every = 1)
print("Save random strategy", flush=True)
scenario_rnd.counts.to_csv("csv/Proximity_N%dK_T%d_s1_ti%d_pz%d_mu%.2f_l%.2f_seed%d_obs%d_rnd_pru%.2lf.csv"%(N/1000,T,t1,N_patient_zero,mu,lamb,seed,n_ranking,pru),
          index=False, sep="\t")
del scenario_rnd

In [ ]:
##### TRACING SCENARIO #######
scenario_trac = Scenario( model, seed=seed+1, 
    ranking_options=dict(ranking=RANKINGS["tracing"], tau=trac_tau),
    observation_options=observation_options,
    intervention_options=intervention_options
)
scenario_trac.run(t_max-t1, print_every = 1)
print("Save tracing strategy", flush=True)
scenario_trac.counts.to_csv("csv/Proximity_N%dK_T%d_s1_ti%d_pz%d_mu%.2f_l%.2f_seed%d_obs%d_trac_t%d_pru%.2lf.csv"%(N/1000,T,t1,N_patient_zero,mu,lamb,seed,n_ranking,trac_tau,pru),
                  index=False, sep="\t")
del scenario_trac